In [5]:
import numpy as np
from collections import defaultdict

# 定义网格世界环境
class GridWorld:
    def __init__(self, grid_size=(4, 4), goal_state=(3, 3), obstacles=None):
        self.grid_size = grid_size
        self.goal_state = goal_state
        self.obstacles = obstacles if obstacles is not None else []
        self.reset()

    def reset(self):
        # 重置智能体位置到随机位置（不是目标位置）
        self.agent_pos = (np.random.randint(0, self.grid_size[0]), np.random.randint(0, self.grid_size[1]))
        while self.agent_pos == self.goal_state:
            self.agent_pos = (np.random.randint(0, self.grid_size[0]), np.random.randint(0, self.grid_size[1]))
        return self.agent_pos

    def step(self, action):
        # 根据动作更新位置
        x, y = self.agent_pos
        if action == 0:  # 上
            new_pos = (max(x - 1, 0), y)
        elif action == 1:  # 下
            new_pos = (min(x + 1, self.grid_size[0] - 1), y)
        elif action == 2:  # 左
            new_pos = (x, max(y - 1, 0))
        elif action == 3:  # 右
            new_pos = (x, min(y + 1, self.grid_size[1] - 1))

        # 检查智能体是否撞到了边界
        if new_pos == self.agent_pos:
            reward = -5  # 撞到边界的较大惩罚
        # 检查新位置是否是障碍物
        elif new_pos in self.obstacles:
            reward = -10  # 障碍物的惩罚
        else:
            reward = -1  # 普通步伐的惩罚
        
        self.agent_pos = new_pos
        
        # 计算奖励，是否结束
        if self.agent_pos == self.goal_state:
            return self.agent_pos, 0, True, {}
        else:
            return self.agent_pos, reward, False, {}

# 定义策略评估和改进过程
def mc_exploring_starts(env, num_episodes, gamma=0.5):
    Q = defaultdict(lambda: np.zeros(4))  # 动作价值函数，Q(s, a)
    returns = defaultdict(list)  # 用于存储每个状态-动作对的回报
    
    for episode_num in range(num_episodes):
        # 1. 随机选择一个起始状态和动作
        state = env.reset()  # 重置环境并随机初始化状态
        action = np.random.choice(4)  # 随机选择动作

        # 2. 生成一个episode
        episode = []
        done = False
        while not done:
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))  # 记录当前状态-动作对和奖励
            state = next_state
            if not done:
                action = np.random.choice(4)  # 继续随机选择下一个动作

        # 3. 计算每个状态-动作对的累积回报
        G = 0
        episode.reverse()  # 从终点往回计算回报
        for (state, action, reward) in episode:
            G = reward + gamma * G  # 累积回报
            # 如果该状态-动作对是第一次在episode中出现
            if (state, action) not in [(x[0], x[1]) for x in episode[:-1]]:
                returns[(state, action)].append(G)
                Q[state][action] = np.mean(returns[(state, action)])  # 更新Q值

    return Q

# 设置障碍物位置
obstacles = [(1, 1), (1, 2), (2, 1)]  # 在网格中定义障碍物

# 创建网格环境并运行算法
env = GridWorld(obstacles=obstacles)
Q = mc_exploring_starts(env, num_episodes=10000)

# 打印学习到的Q值
for state in Q:
    print(f"State {state}: {Q[state]}")


State (3, 0): [-5.43462453 -7.88787263 -7.74505766 -4.49401036]
State (2, 2): [-13.5053108   -2.85072182 -13.489612    -2.62795428]
State (2, 3): [-4.73274213  0.         -4.78161879 -6.36267796]
State (1, 2): [ -4.83660219  -4.30406024 -14.42177459  -3.99343251]
State (1, 3): [ -4.56483688  -2.62703278 -14.03929009  -8.72177692]
State (0, 1): [ -9.13083892 -15.37063874  -4.26058103  -5.31208459]
State (2, 0): [ -5.90809788  -3.78148198  -8.81443465 -13.97005688]
State (3, 2): [-5.31911784 -6.30226099 -4.83691505  0.        ]
State (0, 3): [-7.90423675 -4.69711147 -5.29565487 -7.7546483 ]
State (3, 1): [-13.4925628   -8.34336724  -4.26225767  -2.58334856]
State (1, 0): [ -4.69525411  -5.25744542  -9.51433788 -14.95010369]
State (0, 0): [-8.02371602 -5.38679466 -7.78329406 -5.53661804]
State (2, 1): [-13.94921438  -4.30188935  -4.50517999  -4.44392395]
State (0, 2): [ -9.05696164 -14.07258233  -5.25848897  -4.53445105]
State (1, 1): [ -4.86637994 -12.82543725  -4.89803374 -13.14189311]


In [10]:
import numpy as np
from collections import defaultdict

# 定义网格世界环境
class GridWorld:
    def __init__(self, grid_size=(4, 4), goal_state=(3, 3), obstacles=None, seed=None):
        self.grid_size = grid_size
        self.goal_state = goal_state
        self.obstacles = obstacles if obstacles is not None else []
        self.rng = np.random.default_rng(seed)  # 设置随机数生成器
        self.reset()

    def reset(self):
        # 重置智能体位置到随机位置（不是目标位置或障碍物位置）
        self.agent_pos = (self.rng.integers(0, self.grid_size[0]), self.rng.integers(0, self.grid_size[1]))
        while self.agent_pos == self.goal_state or self.agent_pos in self.obstacles:
            self.agent_pos = (self.rng.integers(0, self.grid_size[0]), self.rng.integers(0, self.grid_size[1]))
        return self.agent_pos

    def step(self, action):
        # 根据动作更新位置
        x, y = self.agent_pos
        if action == 0:  # 上
            new_pos = (max(x - 1, 0), y)
        elif action == 1:  # 下
            new_pos = (min(x + 1, self.grid_size[0] - 1), y)
        elif action == 2:  # 左
            new_pos = (x, max(y - 1, 0))
        elif action == 3:  # 右
            new_pos = (x, min(y + 1, self.grid_size[1] - 1))
        
        # 检查新位置是否是障碍物
        if new_pos in self.obstacles:
            new_pos = self.agent_pos  # 如果是障碍物，位置不变

        self.agent_pos = new_pos
        
        # 计算奖励，是否结束
        if self.agent_pos == self.goal_state:
            return self.agent_pos, 0, True, {}
        else:
            return self.agent_pos, -1, False, {}

# 定义策略评估和改进过程
def mc_exploring_starts(env, num_episodes, gamma=1.0):
    Q = defaultdict(lambda: np.zeros(4))  # 动作价值函数，Q(s, a)
    returns = defaultdict(list)  # 用于存储每个状态-动作对的回报
    
    for episode_num in range(num_episodes):
        # 1. 随机选择一个起始状态和动作
        state = env.reset()  # 重置环境并随机初始化状态
        action = env.rng.integers(0, 4)  # 使用 env 的随机数生成器选择动作

        # 2. 生成一个episode
        episode = []
        done = False
        while not done:
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))  # 记录当前状态-动作对和奖励
            state = next_state
            if not done:
                action = env.rng.integers(0, 4)  # 继续使用 env 的随机数生成器选择下一个动作

        # 3. 计算每个状态-动作对的累积回报
        G = 0
        episode.reverse()  # 从终点往回计算回报
        for (state, action, reward) in episode:
            G = reward + gamma * G  # 累积回报
            # 如果该状态-动作对是第一次在episode中出现
            if (state, action) not in [(x[0], x[1]) for x in episode[:-1]]:
                returns[(state, action)].append(G)
                Q[state][action] = np.mean(returns[(state, action)])  # 更新Q值

    return Q

# 设置障碍物位置
obstacles = [(1, 1), (1, 2), (2, 1)]  # 在网格中定义障碍物
seed = 42  # 设置随机种子

# 创建网格环境并运行算法
env = GridWorld(obstacles=obstacles, seed=seed)
Q = mc_exploring_starts(env, num_episodes=1000)

# 打印学习到的Q值
for state in Q:
    print(f"State {state}: {Q[state]}")


State (0, 2): [-27.         -28.         -32.         -11.66666667]
State (0, 3): [-71.14285714  -9.         -18.8        -67.        ]
State (1, 3): [-54.6         -4.70588235 -22.18181818 -16.42857143]
State (3, 1): [-11.75       -25.38461538 -28.18181818  -4.3       ]
State (0, 1): [-62.66666667 -37.         -31.8        -31.        ]
State (2, 2): [-13.83333333  -8.18181818 -10.1         -4.2       ]
State (2, 3): [-27.25         0.         -21.4         -6.42857143]
State (3, 0): [-37.16666667 -22.33333333 -45.4        -15.42857143]
State (3, 2): [-41.72727273 -25.86666667 -33.78571429   0.        ]
State (1, 0): [-31.66666667 -34.14285714 -29.2        -33.16666667]
State (0, 0): [-41.66666667 -24.         -29.75       -43.        ]
State (2, 0): [-38.85714286 -32.25       -28.875      -37.14285714]


In [8]:

import numpy as np
from collections import defaultdict

# 定义网格世界环境
class GridWorld:
    def __init__(self, grid_size=(4, 4), goal_state=(3, 3), obstacles=None):
        self.grid_size = grid_size
        self.goal_state = goal_state
        self.obstacles = obstacles if obstacles is not None else []
        self.reset()

    def reset(self):
        # 重置智能体位置到随机位置（不是目标位置）
        self.agent_pos = (np.random.randint(0, self.grid_size[0]), np.random.randint(0, self.grid_size[1]))
        while self.agent_pos == self.goal_state:
            self.agent_pos = (np.random.randint(0, self.grid_size[0]), np.random.randint(0, self.grid_size[1]))
        return self.agent_pos

    def step(self, action):
        # 根据动作更新位置
        x, y = self.agent_pos
        if action == 0:  # 上
            new_pos = (max(x - 1, 0), y)
        elif action == 1:  # 下
            new_pos = (min(x + 1, self.grid_size[0] - 1), y)
        elif action == 2:  # 左
            new_pos = (x, max(y - 1, 0))
        elif action == 3:  # 右
            new_pos = (x, min(y + 1, self.grid_size[1] - 1))

        # 检查智能体是否撞到了边界
        if new_pos == self.agent_pos:
            reward = -5  # 撞到边界的较大惩罚
        # 检查新位置是否是障碍物
        elif new_pos in self.obstacles:
            reward = -10  # 障碍物的惩罚
        else:
            reward = -1  # 普通步伐的惩罚
        
        self.agent_pos = new_pos
        
        # 计算奖励，是否结束
        if self.agent_pos == self.goal_state:
            return self.agent_pos, 0, True, {}
        else:
            return self.agent_pos, reward, False, {}

# 定义策略评估和改进过程
def mc_exploring_starts(env, num_episodes, gamma=0.5):
    Q = defaultdict(lambda: np.zeros(4))  # 动作价值函数，Q(s, a)
    returns = defaultdict(list)  # 用于存储每个状态-动作对的回报
    
    for episode_num in range(num_episodes):
        # 1. 随机选择一个起始状态和动作
        state = env.reset()  # 重置环境并随机初始化状态
        action = np.random.choice(4)  # 随机选择动作

        # 2. 生成一个episode
        episode = []
        done = False
        while not done:
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))  # 记录当前状态-动作对和奖励
            state = next_state
            if not done:
                action = np.random.choice(4)  # 继续随机选择下一个动作

        # 3. 计算每个状态-动作对的累积回报
        G = 0
        episode.reverse()  # 从终点往回计算回报
        for (state, action, reward) in episode:
            G = reward + gamma * G  # 累积回报
            # 如果该状态-动作对是第一次在episode中出现
            if (state, action) not in [(x[0], x[1]) for x in episode[:-1]]:
                returns[(state, action)].append(G)
                Q[state][action] = np.mean(returns[(state, action)])  # 更新Q值

    return Q

# 设置随机种子
np.random.seed(42)

# 设置障碍物位置
obstacles = [(1, 1), (1, 2), (2, 1)]  # 在网格中定义障碍物

# 创建网格环境并运行算法
env = GridWorld(obstacles=obstacles)
Q = mc_exploring_starts(env, num_episodes=10000)

# 打印学习到的Q值
for state in Q:
    print(f"State {state}: {Q[state]}")


State (3, 2): [-5.18811656 -6.382313   -4.44656162  0.        ]
State (2, 3): [-4.46147357  0.         -4.95565147 -6.42799568]
State (1, 1): [ -5.38851306 -12.83525575  -4.70723244 -13.42706318]
State (1, 2): [ -4.92258971  -4.21035967 -14.14191317  -4.4832536 ]
State (2, 2): [-13.50277239  -2.92293781 -13.74299767  -2.70311131]
State (1, 3): [ -4.55324445  -2.83959339 -13.47131204  -8.4957274 ]
State (3, 0): [-5.27978989 -7.65986865 -7.8127696  -4.56675991]
State (2, 1): [-14.33709289  -4.65342687  -5.07546044  -4.58292428]
State (0, 1): [ -9.97410203 -15.54891439  -4.58459962  -5.49136428]
State (3, 1): [-13.54117753  -8.76309513  -4.3552792   -2.78991855]
State (0, 0): [-7.89186349 -5.42933711 -7.89438625 -5.68795024]
State (2, 0): [ -5.45146294  -4.48449988  -9.11773605 -13.30314028]
State (0, 3): [-7.72053698 -4.96831547 -5.64628955 -7.75476451]
State (1, 0): [ -4.79095041  -5.15018511  -9.40135088 -15.19555447]
State (0, 2): [ -8.74788674 -13.84959345  -5.26449592  -4.45455017]


In [ ]:
def mc_epsilon_greedy(env, num_episodes, gamma=1.0, epsilon=0.1):
    Q = defaultdict(lambda: np.zeros(4))  # 动作价值函数，Q(s, a)
    returns = defaultdict(list)  # 用于存储每个状态-动作对的回报

    def epsilon_greedy_policy(state, epsilon):
        if np.random.rand() < epsilon:
            return np.random.choice(4)  # 随机选择动作（探索）
        else:
            return np.argmax(Q[state])  # 选择当前估计最优动作（利用）

    for episode_num in range(num_episodes):
        state = env.reset()  # 初始化状态
        episode = []
        done = False

        while not done:
            action = epsilon_greedy_policy(state, epsilon)  # 使用ε-greedy策略选择动作
            next_state, reward, done, _ = env.step(action)  # 执行动作并获得反馈
            episode.append((state, action, reward))  # 记录该步
            state = next_state  # 转移到下一个状态

        # 计算每个状态-动作对的累积回报
        G = 0
        episode.reverse()  # 从最后一步开始回溯
        visited_sa_pairs = set()  # 记录已经更新过的 (state, action) 对

        for (state, action, reward) in episode:
            G = reward + gamma * G  # 计算累计回报
            if (state, action) not in visited_sa_pairs:
                visited_sa_pairs.add((state, action))  # 避免在同一个episode中多次更新同一个状态-动作对
                returns[(state, action)].append(G)
                Q[state][action] = np.mean(returns[(state, action)])  # 更新动作价值函数

    return Q
